### Stanford Paper on LSTM Neural Networks for stock prices volatility prediction

http://cs230.stanford.edu/projects_fall_2019/reports/26254244.pdf

### Tutorial for building an LSTM neural network for time-series prediction

https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

### Importing the required libraries

In [ ]:
import pandas as pd
from pandas.plotting import autocorrelation_plot
from pandas_datareader import data
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

%matplotlib inline

from tensorflow import keras
from tensorflow.keras import layers

# Datetime

import datetime

# Scikit-Learn

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score

# GARCH model

import pyflux as pf

# Keras

from keras.models import Model
from keras.layers import *
from keras.utils.vis_utils import plot_model

# Tensorflow

import tensorflow as tf

### Reading the csv file with the financial data

In [ ]:
df = pd.read_csv(r'input/financial_data.csv')

print (df.head())
print (df.shape)

As we see here, we have 254 columns, corresponding to the 254 business days for which we have financial data and 10 columns, which are the 10 financial indicators we have.

# Data Cleaning

### Transposing the dataframe 

As we are working with time series data, we should have the dates as one column and will thus use transpose() for this.

In [ ]:
df = df.transpose()

print(df.head())
print(df.shape)

### Reset the index of the dataframe

In [ ]:
df = df.reset_index()

print(df.head())

### Renaming the columns with the financial indicators name

In [ ]:
df = df.rename(columns={
    
    df.columns[0]: 'Date',
    df.columns[1]:'Open',
    df.columns[2]: 'Close',
    df.columns[3]:'High',
    df.columns[4]:'Low',
    df.columns[5]: 'Volume',
    df.columns[6]: 'RSI14',
    df.columns[7]:'SMA14',
    df.columns[8]: 'EMA14',
    df.columns[9]:'MACD_sl',
    df.columns[10]:'MACD_h'

})

print (df.head())

### Converting the Date column into a Date

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'], format='%Y%m%d')

### Setting the Date column as the index

In [ ]:
df.set_index('Date', inplace=True)

print(df.head())

### Adding volume from Yahoo Finance API

In [ ]:
start_date = '1990-06-11'
end_date = '2020-06-10'

In [ ]:
panel_data = data.DataReader('^GSPC', 'yahoo', start_date, end_date)

In [ ]:
print (panel_data.head())

print( panel_data.shape)

In [ ]:
df['Volume'] =  panel_data['Volume']

### Function to print out the data type of each column

In [ ]:
def list_columns_to_dropna(df, column_list):
    
    for column in column_list:
        
        df = df[df[column].notna()]
        
    return df

In [ ]:
column_list = ['Open','Close']

df = list_columns_to_dropna(df, column_list)

print (df.head())
print (df.shape)

In [ ]:
def print_data_type_of_dataframe_columns(df):
    
    dataTypeSeries = df.dtypes
 
    print('Data type of each column of Dataframe :')
    print(dataTypeSeries)

# Feature Engineering

## Logarithmic Features

### Log Returns

In [ ]:
df['Log_Returns'] = np.log(df.Close) - np.log(df.Close.shift(1))

print(df.head())
print(df.tail())

### Log Trading Range

In [ ]:
df['Log_Trading_Range'] = np.log(df.High) - np.log(df.Low)

print(df.head())

### Log Volume Change

In [ ]:
df['Log_Volume_Change'] = np.log(df.Volume) - np.log(df.Volume.shift(1))

print(df.head())

## Volatility

### Previous 10-day Volatility

In [ ]:
df['Previous_10_Day_Volatility'] = df['Log_Returns'].rolling(window = 10).std()

print(df.tail())

### Previous 30-day Volatility

In [ ]:
df['Previous_30_Day_Volatility'] = df['Log_Returns'].rolling(window = 30).std()

print(df.head())

### Next 10-days volatility

In [ ]:
df['Next_10_Days_Volatility'] = df['Log_Returns'].iloc[::-1].rolling(window = 10).std().iloc[::-1]

print(df.head())

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.to_csv(r'output/output.csv')

## GARCH 

In [ ]:
## Garch predictions for the entire dataset of SPX

### Building a new dataframe for splitting the dataframe in test and training data

In [ ]:
X = df[df.first_valid_index():df.last_valid_index()- datetime.timedelta(1500)]

print (X.tail())

### Building a GARCH model using PyFlux

In [ ]:
GARCH_model = pf.GARCH(X, target = 'Log_Returns', p=1, q=1)

x = GARCH_model.fit()

x.summary()

### Making rolling predictions using the GARCH Model

In [ ]:
GARCH_rolling_predictions = GARCH_model.predict_is(h = len(X) - 50, fit_once = True)

print(GARCH_rolling_predictions.head())

### Making forward-looking predictions using the GARCH Model

In [ ]:
GARCH_forward_looking_predictions = GARCH_model.predict(h=1500)

print(GARCH_forward_looking_predictions.head())

### Renaming one of the columns of the GARCH Model Dataframe

In [ ]:
GARCH_rolling_predictions.rename(columns={'Series':'GARCH_rolling_predictions'}, inplace =True)

GARCH_forward_looking_predictions.rename(columns={'Log_Returns':'GARCH_forward_looking_predictions'}, inplace =True)

### Adding the new feature to the current dataframe

In [ ]:
df = pd.concat([df, GARCH_rolling_predictions], axis=1)
df = pd.concat([df, GARCH_forward_looking_predictions], axis=1)

### Replacing Nan values with 0s for the GARCH Predictions columns

Rational for this from Keras's creator:

https://stackoverflow.com/questions/52570199/multivariate-lstm-with-missing-values

In [ ]:
df['GARCH_forward_looking_predictions'] =  df['GARCH_forward_looking_predictions'].fillna(0)
df['GARCH_rolling_predictions'] =  df['GARCH_rolling_predictions'].fillna(0)

### Checking the results of our transformations

In [ ]:
print (df.tail())

In [ ]:
# Notes

In [ ]:
# Better to predict VIX prices than realized volatility of SPX

In [ ]:
#Use it to predict VIX

Look at relationship of (5, 10, 30) realized volatility of SPX versus VIX prices (Plot)

Calculate in Excel

### Building a new dataframe for splitting the dataframe in test and training data

### Using dropna on several columns

In [ ]:
def list_columns_to_dropna(df, column_list):
    
    for column in column_list:
        
        df = df[df[column].notna()]
        
    return df

In [ ]:
column_list = ['Open', 'Log_Returns','Previous_10_Day_Volatility','Next_10_Days_Volatility','Previous_30_Day_Volatility']

df = list_columns_to_dropna(df, column_list)

print (df.head())
print (df.shape)

### Exporting the final dataframe to csv

In [ ]:
df.to_csv(r'output/output.csv')

# Exploratory Data Analysis (EDA)

### Plotting out the S&P 500 Prices from 1990 to 2020

In [ ]:
df['Close'].plot(label = 'S&P 500', figsize =(16,8), title = 'S&P 500 Stock Prices from 1990 to 2020')
plt.show()

### Plotting out the 10-days forward looking volatility of  S&P 500 Prices from 1990 to 2020

In [ ]:
df['Next_10_Days_Volatility'].plot(label = 'S&P 500', figsize =(16,8), title = '10-days forward looking volatility of  S&P 500 Prices from 1990 to 2020')
plt.show()

# Feature Selection

### Pearson Correlation Matrix

In [ ]:
def build_pearson_correlation_matrix_of_dataframe(size_x,size_y,dataframe,correlation_target,correlation_minimum_criteria):

    # Using Pearson Correlation

    plt.figure(figsize=(size_x,size_y))
    cor = dataframe.corr()
    sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
    plt.savefig('Images/pearson_correlation_matrix.png', bbox_inches='tight')
    plt.show()

    # Correlation with output variable

    target = abs(cor[correlation_target])

    #Selecting and printing highly correlated features

    relevant_features = target[target>correlation_minimum_criteria]
    print(relevant_features)

In [ ]:
build_pearson_correlation_matrix_of_dataframe(20,20,df,"Next_10_Days_Volatility",0.2)

# Data Preparation

### Splitting the data into train and test sets

In [ ]:
X = np.array(df.drop(["Next_10_Days_Volatility",'Low','High','Close','Open','Volume','MACD_h','MACD_sl','RSI14','SMA14','EMA14'], axis=1).values)
y = np.array(df["Next_10_Days_Volatility"].values).reshape(-1, 1) 

test_size = 1500

X_train = X[test_size:,]
X_test = X[:test_size,]
y_train = y[test_size:]
y_test = y[:test_size]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

### Defining a function to get lagged versions of the features

This function increases the number of features of the dataset by "lagging" every feature.

In [ ]:
def get_lagged(x, y, t, s):
    
    lagged = []
    
    for i in range(x.shape[0] - t):
        
        if i == x.shape[0] - t:
            
            break
            
        for k in range(t):
            
            if k < t:
                
                lagged.append(x[i+k])
                
    lagged = np.array(lagged).reshape(s)
    
    return lagged, y[:lagged.shape[0],]

In [ ]:
N = 30

X_train, y_train = get_lagged(X_train, y_train, N, (X_train.shape[0]-N, N*X_train.shape[1]))
X_test, y_test = get_lagged(X_test, y_test, N, (X_test.shape[0]-N, N*X_test.shape[1]))

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
T = 4

X_train, y_train = get_lagged(X_train, y_train, T, (X_train.shape[0]-T, T, X_train.shape[1]))
X_test, y_test = get_lagged(X_test, y_test, T, (X_test.shape[0]-T, T, X_test.shape[1]))

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

# LSTM

### Building the LSTM Model

In [ ]:
inputLSTM = Input(shape=(X_train.shape[1], X_train.shape[2]))
y = LSTM(200, return_sequences=True)(inputLSTM)
y = LSTM(200)(y)
y = Dense(1)(y)
lstm = Model(inputs=inputLSTM, outputs=y)
lstm.summary()

### Plotting out the LSTM network

In [ ]:
plot_model(lstm, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### Declaring the parameters of the LSTM Model

In [ ]:
lstm.compile(optimizer=keras.optimizers.Adam(lr=0.01),loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.RootMeanSquaredError()])

Get data for SPX where you have Open Close and Volumen (1960)

### Fitting the LSTM Model

In [ ]:
hist = lstm.fit(X_train, y_train,batch_size=700,epochs=60,verbose=1,validation_split=0.3,shuffle=False)

6/6 [==============================] - 1s 85ms/step - loss: 3.8515e-06 - root_mean_squared_error: 0.0020 - val_loss: 4.7232e-06 - val_root_mean_squared_error: 0.0022
Epoch 48/60
6/6 [==============================] - 1s 88ms/step - loss: 3.7894e-06 - root_mean_squared_error: 0.0019 - val_loss: 4.6308e-06 - val_root_mean_squared_error: 0.0022
Epoch 49/60
6/6 [==============================] - 1s 85ms/step - loss: 3.7271e-06 - root_mean_squared_error: 0.0019 - val_loss: 4.5384e-06 - val_root_mean_squared_error: 0.0021
Epoch 50/60
6/6 [==============================] - 1s 84ms/step - loss: 3.6649e-06 - root_mean_squared_error: 0.0019 - val_loss: 4.4461e-06 - val_root_mean_squared_error: 0.0021
Epoch 51/60
6/6 [==============================] - 1s 85ms/step - loss: 3.6026e-06 - root_mean_squared_error: 0.0019 - val_loss: 4.3541e-06 - val_root_mean_squared_error: 0.0021
Epoch 52/60
6/6 [==============================] - 1s 84ms/step - loss: 3.5403e-06 - root_mean_squared_error: 0.0019 - val

### Plotting the RSME for training and validation

In [ ]:
plt.plot(hist.history['root_mean_squared_error'])
plt.plot(hist.history['val_root_mean_squared_error'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

### Printing out the predictions made by the model

In [ ]:
for ind, i in enumerate(lstm.predict(X_test)):
    
    print('Prediction: ' + str('{:.2f}'.format(round(100 * round(i[0], 4),3))) + ',    ' + 'Actual Value: ' + str('{:.2f}'.format(round(100 * round(y_test[ind][0],4),2))))

### Printing out the results of the model

In [ ]:
def printing_out_results_of_a_model(model,X_test,y_test):
    
    y_pred = model.predict(X_test)
    
    # Print the R2 score 

    print ("R2 score:\n") 
    print (('{:.2f}'.format((100*(r2_score(y_test, y_pred))))) + " %")

    print ("\n")
    
    # Print the RMSE

    print ("RMSE:\n")
    print (math.sqrt(mean_squared_error(y_test, y_pred)))
    
    print ('\n')
    
    # Print the mean squared error
    
    print ("Mean Squared Error:\n")
    print (mean_squared_error(y_test, y_pred))

In [ ]:
printing_out_results_of_a_model(lstm, X_test, y_test)